In [ ]:
# This application fetches weather data from a URL, identifies the header row, and loads the data into a pandas DataFrame.
# Author: Dima Kozlovskyy


import pandas as pd
import requests
import re
import matplotlib.pyplot as plt

### Getting Data

In [ ]:
url = "https://cli.fusio.net/cli/climate_data/webdata/hly4935.csv"

In [ ]:
# Geeting ready the data to load into DataFrame by identifying the header row number.

lines = requests.get(url).text.splitlines()

start_header = 0
i = 0

for line in lines:
    commas = re.findall(r',', line)
    if len(commas) > 1:
        start_header = i
        break
    i += 1

print(f"Header starts at line: {start_header}")

In [ ]:
df = pd.read_csv("https://cli.fusio.net/cli/climate_data/webdata/hly4935.csv", skiprows=start_header)

df.columns

In [ ]:

# Plotting temperature data.

temperature_data = df['temp']
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d%H%M')

plt.figure(figsize=(12,5))
plt.plot(df['date'], df['temp'], color='orange')
plt.title('Temperature Data')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.show()